In [1]:
import pandas as pd
import nltk

from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [3]:
# Loading dataset and view a few records.
train_df = pd.read_excel('E:\Python\Machine Learning\datasets\documents_category.xlsx', sheet_name='training')
train_df

,doc_no,content,class_label
0,D1,Saturn Dealer’s Car,Auto
1,D2,Toyota Car Tercel,Auto
2,D3,Baseball Game Play,Sports
3,D4,Pulled Muscle Game,Sports
4,D5,Colored GIFs Root,Computer


In [4]:
test_df = pd.read_excel('E:\MYLEARN\Analytics-1\datasets\documents_category.xlsx', sheet_name='test')
test_df

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\MYLEARN\\Analytics-1\\datasets\\documents_category.xlsx'

In [309]:
# Calculate P(E1|O), P(E2|O), P(E3|O) & P(E4|O)
# Calculate P(O)
# Calculate P(E1), P(E2), P(E3) & P(E4)

# To find out : P(O | E1, E2, E3, E4) 
# = [P(E1|O) * P(E2|O) * P(E3|O) * P(E4|O) / (P(E1), P(E2), P(E3) & P(E4))] * P(O)

In [310]:
# number of distinct values of the outcome (flu column)
number_of_class_labels=len(train_df['class_label'].unique())

In [311]:
# the list of distinct values of the outcome (flu column)
train_df['class_label'].unique()

array(['Auto', 'Sports', 'Computer'], dtype=object)

In [312]:
Count_Row=train_df.shape[0] #gives number of row count in the training df
Count_Col=train_df.shape[1] #gives number of col count in the training df

In [313]:
# create an empty df to store propbailities

class_prob_df = pd.DataFrame(columns=['class_label', 'probability'], index=range(number_of_class_labels))
class_prob_df

,class_label,probability
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN


In [314]:
train_df['class_label'].value_counts()
    

Auto        2
Sports      2
Computer    1
Name: class_label, dtype: int64

In [315]:
i=0
for val, cnt in train_df['class_label'].value_counts().iteritems():
    print ('value', val, 'was found', cnt, 'times')
    class_prob_df.loc[i].class_label = val
    class_prob_df.loc[i].probability = cnt/Count_Row
    i = i +1
    
class_prob_df

value Auto was found 2 times
value Sports was found 2 times
value Computer was found 1 times


,class_label,probability
0,Auto,0.4
1,Sports,0.4
2,Computer,0.2


In [316]:
# get all the 
all_tokens = []
for idx, row in train_df.iterrows():
    for word in word_tokenize(row.content):
        all_tokens.append(word)
    
print(len(all_tokens), all_tokens)


17 ['Saturn', 'Dealer', '’', 's', 'Car', 'Toyota', 'Car', 'Tercel', 'Baseball', 'Game', 'Play', 'Pulled', 'Muscle', 'Game', 'Colored', 'GIFs', 'Root']


In [317]:
all_tokens_unique = set(all_tokens)
print(len(all_tokens_unique), all_tokens_unique)

15 {'Dealer', 'Game', 'Saturn', 'Car', 'Play', 'Colored', 'Root', '’', 'Baseball', 'Muscle', 'GIFs', 'Pulled', 'Tercel', 'Toyota', 's'}


In [318]:
# remove stop words
stop_words = set(stopwords.words('english'))

# filter out stop words
tokens = [w for w in all_tokens_unique if not w in stop_words]
print(len(tokens), tokens)

# keep out the punctuations
tokens = [word for word in tokens if word.isalpha()]
print(len(tokens), tokens)

14 ['Dealer', 'Game', 'Saturn', 'Car', 'Play', 'Colored', 'Root', '’', 'Baseball', 'Muscle', 'GIFs', 'Pulled', 'Tercel', 'Toyota']
13 ['Dealer', 'Game', 'Saturn', 'Car', 'Play', 'Colored', 'Root', 'Baseball', 'Muscle', 'GIFs', 'Pulled', 'Tercel', 'Toyota']


In [319]:
# merge documents for each category
merged_train_df = train_df.groupby('class_label')['content'].apply(' '.join).reset_index()

merged_train_df

,class_label,content
0,Auto,Saturn Dealer’s Car Toyota Car Tercel
1,Computer,Colored GIFs Root
2,Sports,Baseball Game Play Pulled Muscle Game


In [320]:
# pre - process each merged document
for idx, row in merged_train_df.iterrows():
    
    # tokenize & unique words
    temp1_tokens = []
    for word in word_tokenize(row.content):
        temp1_tokens.append(word)
        #temp1_tokens = set(temp1_tokens)
        
    #print(temp1_tokens)
    
    # remove stop words
    temp2_tokens = []
    for word in temp1_tokens:
        if not word in stop_words:
            temp2_tokens.append(word)
            
    #print(temp2_tokens)
    
    # remove punctuations
    temp3_tokens = []
    for word in temp2_tokens:
        if word.isalpha():
            temp3_tokens.append(word)
            
    #print(temp3_tokens)
    
    temp4_tokens = " ".join(temp3_tokens)
    #print(temp4_tokens)
    
    merged_train_df.at[idx, 'content'] = temp4_tokens
    merged_train_df.at[idx, 'no_of_words_in_category'] = len(temp3_tokens)

In [321]:
# Category wise merged content
merged_train_df

,class_label,content,no_of_words_in_category
0,Auto,Saturn Dealer Car Toyota Car Tercel,6.0
1,Computer,Colored GIFs Root,3.0
2,Sports,Baseball Game Play Pulled Muscle Game,6.0


In [326]:
# merge to get catgory probability
# merged_train_df
# class_prob_df
merged_train_df= pd.merge(merged_train_df, class_prob_df[['class_label', 'probability']], on='class_label')

In [327]:
merged_train_df

,class_label,content,no_of_words_in_category,probability
0,Auto,Saturn Dealer Car Toyota Car Tercel,6.0,0.4
1,Computer,Colored GIFs Root,3.0,0.2
2,Sports,Baseball Game Play Pulled Muscle Game,6.0,0.4


In [328]:
final_df = pd.DataFrame()

row_counter = 0

for idx, row in merged_train_df.iterrows():
    for token in tokens:
        # find the number of occurances of the token in the current category of documents
        no_of_occurances = row.content.count(token)
        no_of_words_in_category = row.no_of_words_in_category
        no_unique_words_all = len(tokens)
        
        prob_of_token = (no_of_occurances+ 1)/ (no_of_words_in_category+ no_unique_words_all)
        #print(row.class_label, token, no_of_occurances, prob_of_token)
        final_df.at[row_counter, 'class_label'] = row.class_label
        final_df.at[row_counter, 'token'] = token
        final_df.at[row_counter, 'no_of_occurances'] = no_of_occurances
        final_df.at[row_counter, 'no_of_words_in_category'] = no_of_words_in_category
        final_df.at[row_counter, 'no_unique_words_all'] = no_unique_words_all
        final_df.at[row_counter, 'prob_of_token_category'] = prob_of_token
        
        row_counter = row_counter + 1

In [374]:
final_df

,class_label,token,no_of_occurances,no_of_words_in_category,no_unique_words_all,prob_of_token_category
0,Auto,Dealer,1.0,6.0,13.0,0.105263
1,Auto,Game,0.0,6.0,13.0,0.052632
2,Auto,Saturn,1.0,6.0,13.0,0.105263
3,Auto,Car,2.0,6.0,13.0,0.157895
4,Auto,Play,0.0,6.0,13.0,0.052632
5,Auto,Colored,0.0,6.0,13.0,0.052632
6,Auto,Root,0.0,6.0,13.0,0.052632
7,Auto,Baseball,0.0,6.0,13.0,0.052632
8,Auto,Muscle,0.0,6.0,13.0,0.052632
9,Auto,GIFs,0.0,6.0,13.0,0.052632


In [378]:
# Calculate P(Category/Document) 
#      = P(Category) * P(Word1/Category) * P(Word2/Category) * P(Word3/Category)

# P(Auto/D6) = P(Auto) * P(Engine/Auto) * P(Noises/Auto) * P(Car/Auto)
for idx, row in test_df.iterrows():
    
    # tokenize & unique words
    temp1_tokens = []
    for word in word_tokenize(row.content):
        temp1_tokens.append(word)
        #temp1_tokens = set(temp1_tokens)
        
    # remove stop words
    temp2_tokens = []
    for word in temp1_tokens:
        if not word in stop_words:
            temp2_tokens.append(word)
          
    # remove punctuations
    temp3_tokens = []
    for word in temp2_tokens:
        if word.isalpha():
            temp3_tokens.append(word)
            
    #temp4_tokens = " ".join(temp3_tokens)
    #print(temp4_tokens)
    
    prob = 1 
    
    # process for each class_label
    for idx1, row1 in merged_train_df.iterrows():
        print("class: "+ row1.class_label)
        for token in temp3_tokens:
            # find the token in final_df for the given category, get the probability
            # row1.class_label & token
        
            print("      : "+ token)  
        
            temp_df = final_df[(final_df['class_label'] == row1.class_label) & (final_df['token'] == token)]

            # process for exception
            if (temp_df.shape[0] == 0):
                token_prob = 1/(row1.no_of_words_in_category+ no_unique_words_all)
                print("       no token found prob :", token_prob)
                prob = prob * token_prob
            else:
                token_prob = temp_df.get_value(temp_df.index[0],'prob_of_token_category')
                print("       token prob          :", token_prob)
                prob = prob * token_prob

            prob = prob * row1.probability

        col_at = 'prob_'+row1.class_label

        test_df.at[idx, col_at] = prob


test_df

class: Auto
      : Home
       no token found prob : 0.05263157894736842
      : Runs
       no token found prob : 0.05263157894736842
      : Game
       token prob          : 0.0526315789474
class: Computer
      : Home
       no token found prob : 0.0625
      : Runs
       no token found prob : 0.0625
      : Game
       token prob          : 0.0625
class: Sports
      : Home
       no token found prob : 0.05263157894736842
      : Runs
       no token found prob : 0.05263157894736842
      : Game
       token prob          : 0.157894736842
class: Auto
      : Car
       token prob          : 0.157894736842
      : Engine
       no token found prob : 0.05263157894736842
      : Noises
       no token found prob : 0.05263157894736842
class: Computer
      : Car
       token prob          : 0.0625
      : Engine
       no token found prob : 0.0625
      : Noises
       no token found prob : 0.0625
class: Sports
      : Car
       token prob          : 0.0526315789474
      : Engine


,doc_no,content,class_label,prob_Sports,prob_Auto,prob_Computer
0,D6,Home Runs Game,NaN,5.101403e-16,0.000009,1.822423e-11
1,D7,Car Engine Noises,NaN,5.101403e-16,0.000028,5.467269e-11
